In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install transformers tqdm pandas nltk rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=632d5b41418c07d1802e469ddf541faa464633ef4760963bac7c4262e6156990
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
!pip install torch transformers

In [ ]:
!pip install peft transformers

In [ ]:
import tqdm
import transformers
from transformers import AutoTokenizer, T5ForConditionalGeneration

#Load X-Sum

In [ ]:
from datasets import load_dataset

# Load the XSum dataset with trust_remote_code=True
dataset = load_dataset("xsum", trust_remote_code=True)

# Display the dataset (you can access it as a dictionary of splits)
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

(…)SUM-EMNLP18-Summary-Data-Original.tar.gz:   0%|          | 0.00/255M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})


In [ ]:
from datasets import load_dataset
import pandas as pd
from IPython.display import display

dataset = load_dataset("xsum", trust_remote_code=True)

train_df = dataset['train'].to_pandas()

display(train_df.head())


,document,summary,id
0,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,35232142
1,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,40143035
2,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,35951548
3,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer carried o...,36266422
4,Patients and staff were evacuated from Cerahpa...,An armed man who locked himself into a room at...,38826984


In [ ]:
from datasets import load_dataset
from datasets import DatasetDict

train_subset = dataset["train"].shuffle(seed=42).select(range(200))
val_subset = dataset["validation"].shuffle(seed=42).select(range(200))
test_subset = dataset["test"].shuffle(seed=42).select(range(200))

dataset = DatasetDict({
    "train": train_subset,
    "validation": val_subset,
    "test": test_subset
})

## T5-Small model##

- Prompt Engineering provides the best coverage and coherence but heavily depends on the quality of the prompt.
- Prompt Tuning enhances flexibility with adjusted prompts, though the gains may be limited.
- PEFT LoRA is more resource-efficient and suitable for large tasks but requires fine-tuning to avoid errors like repetitions.

In [ ]:
import pandas as pd
import nltk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
from rouge_score import rouge_scorer
from peft import LoraConfig, get_peft_model, TaskType

nltk.download('punkt')

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

##Fine tuning##

fine-tuning technique number 1:Prompt TUNING

In [ ]:
import torch
import pandas as pd
import nltk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
from tqdm import tqdm
from rouge_score import rouge_scorer
from torch import nn


model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Freeze all the model parameters except the embeddings
for param in model.parameters():
    param.requires_grad = False

# Let's introduce learnable prompt tokens.
class PromptTuning(nn.Module):
    def __init__(self, model, prompt_length=5):
        super().__init__()
        self.model = model
        self.prompt_embeddings = nn.Parameter(torch.randn(1, prompt_length, model.config.d_model))

    def forward(self, input_ids):
        prompt = self.prompt_embeddings.expand(input_ids.size(0), -1, -1)
        inputs_embeds = self.model.get_input_embeddings()(input_ids)
        combined_inputs = torch.cat([prompt, inputs_embeds], dim=1)
        return self.model.generate(inputs_embeds=combined_inputs)

# Add the prompt tuning to the model
prompt_tuning_model = PromptTuning(model)

optimizer = AdamW(prompt_tuning_model.parameters(), lr=1e-5)

def prompt_tuning_summary(text):
    input_text = f"Generate a concise yet comprehensive summary of this article, highlighting the key events, important quotes, and the main objective of the military action: {text}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    generated = prompt_tuning_model(inputs)
    return tokenizer.decode(generated[0], skip_special_tokens=True)

train_df_subset['prompt_tuning_result'] = [
    prompt_tuning_summary(text) for text in tqdm(train_subset['document'], desc="Prompt Tuning Progress")
]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Prompt Tuning Progress:   0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Prompt Tuning Progress: 100%|██████████| 200/200 [03:09<00:00,  1.06it/s]


In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def compute_rouge(reference, generated):
    scores = scorer.score(reference, generated)
    return {
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure
    }

# Apply to each row for prompt_tuning_result
rouge_scores_prompt_tuning = train_df_subset.apply(
    lambda row: compute_rouge(row['summary'], row['prompt_tuning_result']),
    axis=1
)

# Convert the list of dictionaries to a DataFrame for better readability
rouge_df_prompt_tuning = pd.DataFrame(list(rouge_scores_prompt_tuning))

mean_rouge_prompt_tuning = rouge_df_prompt_tuning.mean()

print("Mean ROUGE Scores for Prompt Tuning:")
print(mean_rouge_prompt_tuning)

Mean ROUGE Scores for Prompt Tuning:
rouge1    0.110629
rouge2    0.003690
rougeL    0.098479
dtype: float64


fine-tuning technique number 2:Prompt ENGINEERING

In [ ]:
import torch
import pandas as pd
import nltk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
from tqdm import tqdm
from rouge_score import rouge_scorer
from torch import nn

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Freeze all model parameters
for param in model.parameters():
    param.requires_grad = False

# Define Prompt Tuning Model with Learnable Embeddings
class PromptTuningModel(nn.Module):
    def __init__(self, model, prompt_length=5):
        super().__init__()
        self.model = model
        self.prompt_embeddings = nn.Parameter(torch.randn(1, prompt_length, model.config.d_model))

    def forward(self, input_ids):
        prompt = self.prompt_embeddings.expand(input_ids.size(0), -1, -1)
        inputs_embeds = self.model.get_input_embeddings()(input_ids)
        combined_inputs = torch.cat([prompt, inputs_embeds], dim=1)
        return self.model.generate(inputs_embeds=combined_inputs)

# Initialize the model with learnable prompt embeddings
prompt_tuning_model = PromptTuningModel(model)

optimizer = AdamW(prompt_tuning_model.parameters(), lr=1e-5)

def prompt_engineering_summary(text):
    input_text = f"Generate a detailed yet concise summary of this article, emphasizing key points, important arguments, and relevant quotes: {text}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate output using the prompt-tuned model
    generated = prompt_tuning_model(inputs)
    return tokenizer.decode(generated[0], skip_special_tokens=True)


train_df_subset['prompt_engineering_result'] = [
    prompt_engineering_summary(text) for text in tqdm(train_subset['document'], desc="Prompt Engineering Progress")
]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Prompt Engineering Progress: 100%|██████████| 200/200 [02:54<00:00,  1.15it/s]


In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)


rouge_scores_prompt_engineering = train_df_subset.apply(
    lambda row: compute_rouge(row['summary'], row['prompt_engineering_result']),
    axis=1
)

rouge_df_prompt_engineering = pd.DataFrame(list(rouge_scores_prompt_engineering))

mean_rouge_prompt_engineering = rouge_df_prompt_engineering.mean()
print("Mean ROUGE Scores for Prompt Engineering:")
print(mean_rouge_prompt_engineering)

Mean ROUGE Scores for Prompt Engineering:
rouge1    0.102444
rouge2    0.006167
rougeL    0.083911
dtype: float64


fine-tuning technique number 3: Lora

In [ ]:
import torch
import pandas as pd
import nltk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
from rouge_score import rouge_scorer
from peft import LoraConfig, get_peft_model, TaskType

nltk.download('punkt')

# Model setup
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = get_peft_model(model, lora_config)

# Freeze all layers except the LoRA parameters
for name, param in model.named_parameters():
    if "lora" not in name:
        param.requires_grad = False

def lora_summary(text):
    input_text = f"Generate a concise and informative summary: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")

    outputs = model.generate(
        **inputs,
        max_length=150,
        min_length=50,
        length_penalty=2.5,
        num_beams=6,
        temperature=0.6,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

train_df_subset['lora_result'] = [
    lora_summary(text) for text in tqdm(train_subset['document'], desc="Generating Summaries with LoRA")
]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Generating Summaries with LoRA:   0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Generating Summaries with LoRA: 100%|██████████| 200/200 [20:19<00:00,  6.10s/it]


In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Apply to each row for lora_result
rouge_scores_lora = train_df_subset.apply(
    lambda row: compute_rouge(row['summary'], row['lora_result']),
    axis=1
)

rouge_df_lora = pd.DataFrame(list(rouge_scores_lora))

mean_rouge_lora = rouge_df_lora.mean()

print("Mean ROUGE Scores for LoRA:")
print(mean_rouge_lora)

Mean ROUGE Scores for LoRA:
rouge1    0.185063
rouge2    0.028279
rougeL    0.127000
dtype: float64


Mean ROUGE Scores for Prompt Tuning:
rouge1    0.110629
rouge2    0.003690
rougeL    0.098479
dtype: float64

Mean ROUGE Scores for Prompt Engineering:
rouge1    0.102444
rouge2    0.006167
rougeL    0.083911
dtype: float64


Mean ROUGE Scores for LoRA:
rouge1    0.185063
rouge2    0.028279
rougeL    0.127000
dtype: float64

#LoRA >  Prompt Tuning > Prompt Engineering